# Elastic query demo

If you have not already executed the prep notebooks for the demo you will need to do so prior to executing this code. The following code is intended to be run against WideWorldDW\_1. The notebook is designed to be fully re-executed including the cleanup components in the first few steps. This will allow you the maximum flexibility when working with this notebook.

# Clean up the demo environment

The demo scripts need a clean database environment to run in. This involves removing tables that would have been created during the demo process. The following code cell will remove those tables if they exist in preparation for rerunning the rest of the environment.

There are three code cells included below which clean up items that will be created later on in the demo:

- The first code cell will remove external tables if they exist
- the second code cell will remove the data sources if they exist
- The third code cell will remove any credentials if they exist

In [1]:
-- remove tables

if exists(select * from sys.external_tables where name = 'Fact_Purchase')
  drop external table dbo.Fact_Purchase;
go
if exists(select * from sys.external_tables where name = 'dimDate')
  drop external table dbo.dimDate;
go
if exists(select * from sys.external_tables where name = 'dimStockItem')
  drop external table dbo.dimStockItem;
go
if exists(select * from sys.external_tables where name = 'dimSupplier')
  drop external table dbo.dimSupplier;
go

In [2]:
-- remove data sources
if exists(select * from sys.external_data_sources where name = 'dsWideWorldDW_2')
  drop external data source dsWideWorldDW_2;
go
if exists(select * from sys.external_data_sources where name = 'dsWideWorldDW_3')
  drop external data source dsWideWorldDW_3;
go

In [3]:
-- remove credentials
if exists(select * from sys.database_scoped_credentials where name = 'elastic')
  drop database scoped credential elastic;
 go

Create a scoped credential which matches a user with read access on the target external databases 

NOTE: Currently AAD accounts are not supported for elastic queries

In [4]:
create database scoped credential elastic with identity = 'elasticuser', secret = 'PassMn2020!';
go


Next we create the external data source, we will create two as we will be working across two databases 
You will need to update the server name to the server name you are using in azure in the following code block

In [5]:
create external data source dsWideWorldDW_2
  with
	(
		type=RDBMS,
		location='<<servername>>.database.windows.net',
		database_name = 'WideWorldDW_2',
		credential = elastic
	)
;

create external data source dsWideWorldDW_3
  with
	(
		type=RDBMS,
		location='<<servername>>.database.windows.net',
		database_name = 'WideWorldDW_3',
		credential = elastic
	)
;


 Next we create the tables we want to connect to

In [6]:

create external table dbo.Fact_Purchase
(
	[Purchase Key] bigint NOT NULL,
	[Date Key] date NOT NULL,
	[Supplier Key] int NOT NULL,
	[Stock Item Key] int NOT NULL,
	[WWI Purchase Order ID] int NULL,
	[Ordered Outers] int NOT NULL,
	[Ordered Quantity] int NOT NULL,
	[Received Outers] int NOT NULL,
	[Package] nvarchar(50) NOT NULL,
	[Is Order Finalized] bit NOT NULL
	--,
	--[Lineage Key] int NOT NULL
	)
	with (DATA_SOURCE = dsWideWorldDW_2)
;

Let's test the new external table and see the results.

In [7]:

-- test the table
select count(*) from dbo.Fact_Purchase;

Create the date dimension table

In [8]:

create external table [dbo].[dimDate](
	[Date] [date] NOT NULL,
	[Day Number] [int] NOT NULL,
	[Day] [nvarchar](10) NOT NULL,
	[Month] [nvarchar](10) NOT NULL,
	[Short Month] [nvarchar](3) NOT NULL,
	[Calendar Month Number] [int] NOT NULL,
	[Calendar Month Label] [nvarchar](20) NOT NULL,
	[Calendar Year] [int] NOT NULL,
	[Calendar Year Label] [nvarchar](10) NOT NULL,
	[Fiscal Month Number] [int] NOT NULL,
	[Fiscal Month Label] [nvarchar](20) NOT NULL,
	[Fiscal Year] [int] NOT NULL,
	[Fiscal Year Label] [nvarchar](10) NOT NULL,
	[ISO Week Number] [int] NOT NULL
	) 
	with (data_source = dsWideWorldDW_3)
; 
go


Create the stock item table. Notice how we need to comment out the photo as that is not supported in elastic query.

In [9]:

create external table [dbo].[dimStockItem](
	[Stock Item Key] [int] NOT NULL,
	[WWI Stock Item ID] [int] NOT NULL,
	[Stock Item] [nvarchar](100) NOT NULL,
	[Color] [nvarchar](20) NOT NULL,
	[Selling Package] [nvarchar](50) NOT NULL,
	[Buying Package] [nvarchar](50) NOT NULL,
	[Brand] [nvarchar](50) NOT NULL,
	[Size] [nvarchar](20) NOT NULL,
	[Lead Time Days] [int] NOT NULL,
	[Quantity Per Outer] [int] NOT NULL,
	[Is Chiller Stock] [bit] NOT NULL,
	[Barcode] [nvarchar](50) NULL,
	[Tax Rate] [decimal](18, 3) NOT NULL,
	[Unit Price] [decimal](18, 2) NOT NULL,
	[Recommended Retail Price] [decimal](18, 2) NULL,
	[Typical Weight Per Unit] [decimal](18, 3) NOT NULL,
--	[Photo] [varbinary](max) NULL,  -- blob/binary types not supported with the exception of varchar/nvarchar(max)
	[Valid From] [datetime2](7) NOT NULL,
	[Valid To] [datetime2](7) NOT NULL,
	[Lineage Key] [int] NOT NULL
	)
	with (data_source = dsWideWorldDW_3)
;
go


The final table we're going to create is the supplier dimension table. We will use this table to illustrate how we don't have to bring all fields over in our elastic query. These fields will still exist in the source table but will not be accessible in the external table created with elastic query.

In [10]:

-- remove some of the available columns
create external table [dbo].[dimSupplier](
	[Supplier Key] [int] NOT NULL,
	[WWI Supplier ID] [int] NOT NULL,
	[Supplier] [nvarchar](100) NOT NULL,
	[Category] [nvarchar](50) NOT NULL,
	--[Primary Contact] [nvarchar](50) NOT NULL,
	--[Supplier Reference] [nvarchar](20) NULL,
	--[Payment Days] [int] NOT NULL,
	[Postal Code] [nvarchar](10) NOT NULL
	--,
--	[Valid From] [datetime2](7) NOT NULL,
--	[Valid To] [datetime2](7) NOT NULL,
--	[Lineage Key] [int] NOT NULL
	)
	with (data_source = dsWideWorldDW_3)

;

go



As you can see in the select below, the columns we do not add to the external table will not show up in the column list for the external table.

In [11]:
-- columns removed from external table
select * from dbo.dimSupplier;

The following query attempts to pull a field that was not brought over at the external table and will result in an error.

In [12]:
-- cannot be pulled through, this results in an error
select [Payment Days] from dbo.dimSupplier;


This query example joins data from both of the databases used in the elastic query external tables. This illustrates the process of data virtualization which allows you to pull data from multiple databases bring them together into one result set using elastic query.

In [13]:

----- complex test

select sum(f.[Ordered Quantity]) as [Total Quantity]
	, si.Brand
	, si.Color
	, su.Supplier
	, su.Category
	, d.[Calendar Year]
from dbo.Fact_Purchase f
	inner join dbo.dimSupplier su on su.[Supplier Key] = f.[Supplier Key]
	inner join dbo.dimStockItem si on si.[Stock Item Key] = f.[Stock Item Key]
	inner join dbo.dimDate d on d.[Date] = f.[Date Key]
where si.Color not like 'N/A'
group by si.Brand
	, si.Color
	, su.Supplier
	, su.Category
	, d.[Calendar Year]
;
go


You should not use external tables as a way to obscure columns for security reasons. The security is established via the credential used to set up the data source. Because of this, we can access the full table by using a remote execution query with the data source. It is recommended that you hide columns to simplify the data set not to secure it.

In [14]:
-- you can execute T-SQL directly as well with the sp_execute_remote; It uses the permissions of the credential

exec sp_execute_remote N'dsWideWorldDW_3', N'select * from dbo.dimSupplier';
go


This remote query shows access to the other database use in our demonstration.

In [15]:
exec sp_execute_remote N'dsWideWorldDW_2', N'select * from dbo.Fact_Purchase';
go
